In [1]:
import requests, time, math, threading
import sqlite3, os
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from queue import Queue
from selenium_tools import get_chrome, find_element, get_soup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [6]:
url = "https://www.google.com/maps/"
datas = pd.read_csv("Taipei_to_hots_copy1.csv", encoding = "utf-8-sig")
# datas = pd.read_csv("NewTaipei_to_hots_copy1.csv", encoding = "utf-8-sig")
print(datas)

     Unnamed: 0.1  Unnamed: 0   ID            shopName  \
0               0           0    0               紫明溪步道   
1               1           1    1           冷水坑公共溫泉浴室   
2               2           2    2  中國信託文薈館CTBC Museum   
3               3           3    3                大湖公園   
4               4           4    4                和興炭坑   
..            ...         ...  ...                 ...   
150           150         150  150     璐露野生活Luluyelife   
151           151         151  151          花博公園（新生園區）   
152           152         152  152             劍潭山親山步道   
153           153         153  153            臺北市兒童新樂園   
154           154         154  154      華山1914文化創意產業園區   

                        shopAddress          shopPhone  \
0            台北市北投區中興路12號附近（陽明書屋附近）       02-2759-3001   
1                 台北市士林區菁山路101巷175號       02-2861-0036   
2    台北市南港區經貿二路168號1樓（中國信託金融園區A棟1樓）  02-3327-7777#1099   
3                    台北市內湖區成功路五段31號  02-3327-7777#1099   
4            

In [7]:
def for_comments(name):
    while True:
        chrome = get_chrome(url, hide = True)
        if "yZqPAf" in str(chrome.page_source):
            chrome = get_chrome(url, hide = True)
        else:
            break
    chrome.refresh()
    #網頁最大化
    chrome.maximize_window()
    xpath1 = "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input"
    element = find_element(chrome, xpath = xpath1)
    element.clear()
    element.click()
    element.send_keys(name + "\n")
    time.sleep(2)
    soup = BeautifulSoup(chrome.page_source, "html.parser")
    if "RZ66Rb" not in str(chrome.page_source):
        url_new = soup.find("a", class_ = "hfpxzc").get("href")            
        chrome.quit()
        chrome = get_chrome(url_new, hide = False)
        time.sleep(0.5)
        #網頁最大化
        chrome.maximize_window()
        chrome.refresh()    
    elif "RZ66Rb" in str(chrome.page_source):
        time.sleep(0.5)
    soup = BeautifulSoup(chrome.page_source, "html.parser") 
    com_list = [name]
    if "Gpq6kf fontTitleSmall" in str(chrome.page_source):
        try:
            try:
                xpath = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()                
            except Exception as e:
                xpath = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[3]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()            
        except Exception as e:
            try:
                xpath = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()                
            except Exception as e:
                xpath = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[3]"
                element = chrome.find_element(By.XPATH, xpath)
                element.click()      
        time.sleep(0.5)
        try:
            try: 
                comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)                     
        except Exception as e:
            try: 
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)   
        time.sleep(0.5)
        try:
            try: 
                comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)                     
        except Exception as e:
            try: 
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)  
        time.sleep(0.5)
        soup2 = BeautifulSoup(chrome.page_source, "html.parser")
        comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
        time.sleep(2)
        for i in comments:
            if "w8nwRe" in str(i): 
                element = chrome.find_element(By.CLASS_NAME, 'w8nwRe')
                element.click()
        time.sleep(0.5)
        try:
            try: 
                comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)                     
        except Exception as e:
            try: 
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"                
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
            except Exception as e:
                comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]"
                pane1 = chrome.find_element(By.XPATH, comment_roller)
                chrome.execute_script("arguments[0].scrollTop = 0", pane1)
        time.sleep(0.5)
        soup2 = BeautifulSoup(chrome.page_source, "html.parser")
        comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
        chrome.quit()
        print(name + "SuccessFul!")
        for com in comments:
            if "wiI7pd" in str(com):
                com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + com.find("span", class_ = "wiI7pd").text])
            else:
                com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + "很讚"])
    else:
        com_list = [name, "暫無評價"]      
    return com_list

In [14]:
try:
    while True:
        chrome = get_chrome(url, hide = False)
        if "yZqPAf" in str(chrome.page_source):
            chrome = get_chrome(url, hide = False)
        else:
            break
    chrome.refresh()
    #網頁最大化
    chrome.maximize_window()
    xpath1 = "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input"
    element = find_element(chrome, xpath = xpath1)
    element.clear()
    element.click()
    element.send_keys(datas["shopName"][0] + "\n")
except Exception as e:
    print(e)

In [15]:
chrome.quit()

#### 有選項

In [2]:
try:
    if "Nv2PK Q2HXcd THOPZb" in str(chrome.page_source):
        soup = BeautifulSoup(chrome.page_source, "html.parser")
        num = -1
        for number in soup.find_all("div", class_ = "Nv2PK Q2HXcd THOPZb"):
            number_text = eval(number.find("span", class_ = "UY7F9").text)
            if number_text > num:
                num = number_text
                url_new = number.find("a", class_ = "hfpxzc").get("href")
        chrome.quit()
        chrome = get_chrome(url_new, hide = False)
        time.sleep(0.5)
        #網頁最大化
        chrome.maximize_window()
        chrome.refresh()    
    elif "RZ66Rb" in str(chrome.page_source):
        time.sleep(0.5)
    try:
        xpath = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
        element = chrome.find_element(By.XPATH, xpath)
        element.click()
    except Exception as e:       
        xpath = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]"
        element = chrome.find_element(By.XPATH, xpath)
        element.click()
    time.sleep(0.5)
    try:        
        comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
    except Exception as e:
        comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)   
    time.sleep(0.5)
    try:        
        comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1)
    except Exception as e:
        comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1) 
    time.sleep(0.5)
    soup2 = BeautifulSoup(chrome.page_source, "html.parser")
    comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
    time.sleep(2)
    for i in comments:
        if "w8nwRe" in str(i): 
            element = chrome.find_element(By.CLASS_NAME, 'w8nwRe')
            element.click()
    time.sleep(0.5)
    try:        
        comment_roller = "/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1)
    except Exception as e:
        comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"    
        pane1 = chrome.find_element(By.XPATH, comment_roller)
        chrome.execute_script("arguments[0].scrollTop = 0", pane1) 
    time.sleep(0.5)
    
    soup2 = BeautifulSoup(chrome.page_source, "html.parser")
    comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
    print(len(comments))
    chrome.quit()
#     print(comments)
    com_list = [name]
    for com in comments:
        if "wiI7pd" in str(com):
            com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + com.find("span", class_ = "wiI7pd").text])
        else:
            com_list.extend(["姓名: " + com.find(class_ = "d4r55").text, "評論: " + "很讚"])
    print(pd.DataFrame(com_list))    
except Exception as e:
    print(e)

name 'chrome' is not defined


In [23]:
try:
    all_comments = []
    for name in datas["shopName"]:        
        while True:
            try:
                com_list = for_comments(name)
                if com_list[2:] == []:
                    com_list = for_comments(name)
            except Exception as e:
                chrome.quit()
                com_list = for_comments(name)
                if com_list[2:] == []:
                    com_list = for_comments(name)
            else:
                break
        all_comments.append(com_list)
        time.sleep(2)
    print(pd.DataFrame(all_comments))          
except Exception as e:
    print(e)

千手千眼觀世音菩薩聖像SuccessFul!
康誥坑溪櫻花大道SuccessFul!
玩聚窩玩具銀行SuccessFul!
烏來龍門精緻溫泉民宿SuccessFul!
烏來龍門精緻溫泉民宿SuccessFul!
世界宗教博物館SuccessFul!
楓樹河濱公園SuccessFul!
益品書屋（淡水館）SuccessFul!
五分山步道SuccessFul!
三角埔頂山（羌子寮山）SuccessFul!
古鐘樓公園SuccessFul!
夥房SuccessFul!
釉藥堂SuccessFul!
三峽安溪路河堤步道SuccessFul!
皇帝殿登山步道SuccessFul!
集順廟SuccessFul!
淡蘭古道石碇段外按古道SuccessFul!
土虱頭觀景臺SuccessFul!
土虱頭觀景臺SuccessFul!
朱銘美術館Juming MuseumSuccessFul!
新北市立十三行博物館SuccessFul!
新北市立淡水古蹟博物館SuccessFul!
朝宗公園SuccessFul!
五股水碓觀景公園SuccessFul!
福隆山登山步道SuccessFul!
五股獅子頭山水匯集廣場SuccessFul!
茶山步道SuccessFul!
台灣山豬城SuccessFul!
老公崎步道SuccessFul!
新寮休閒步道SuccessFul!
紅樹林生態步道SuccessFul!
內洞國家森林遊樂區SuccessFul!
淺沐道休閒農莊SuccessFul!
鶯歌陶瓷博物館水廣場SuccessFul!
塭仔底溼地公園SuccessFul!
鳳祥公園SuccessFul!
三民公園SuccessFul!
安和景觀公園SuccessFul!
林口社區運動公園（小熊公園）SuccessFul!
錦和運動公園SuccessFul!
員山公園－遊逸之丘SuccessFul!
頭前運動公園SuccessFul!
淡水公23公園SuccessFul!
觀音台步道SuccessFul!
新北市坪林茶業博物館SuccessFul!
雙溪清水坑SuccessFul!
虎豹潭SuccessFul!
雙溪平林休閒農場SuccessFul!
雙溪平林休閒農場SuccessFul!
雙溪荷花園SuccessFul!
東和戲院遺址SuccessFul!
不厭亭SuccessFul!
星光

In [83]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 41 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       143 non-null    object
 1   1       140 non-null    object
 2   2       140 non-null    object
 3   3       140 non-null    object
 4   4       140 non-null    object
 5   5       139 non-null    object
 6   6       139 non-null    object
 7   7       138 non-null    object
 8   8       138 non-null    object
 9   9       138 non-null    object
 10  10      138 non-null    object
 11  11      137 non-null    object
 12  12      137 non-null    object
 13  13      137 non-null    object
 14  14      137 non-null    object
 15  15      137 non-null    object
 16  16      137 non-null    object
 17  17      137 non-null    object
 18  18      137 non-null    object
 19  19      137 non-null    object
 20  20      137 non-null    object
 21  21      137 non-null    object
 22  22      137 non-null    ob

In [24]:
df1 = pd.DataFrame(all_comments)
df1

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,姓名: Jimmy Chang,評論: 在山坡之中竟藏著一座千手觀音佛像！但佛像部分仍未完工，僅開放六、日及國定假日可以進入...,姓名: 依娜,評論: 終於有緣份進來看千手觀音神像了，這工程太耗大到現在還在蓋，大殿可以進去參觀拍照，牆四...,姓名: Steven Hsu,評論: 一個令人驚嘆又讓人感受非常舒服的地方，在金氏世界紀錄的見證，雄偉觀音菩薩聖像在陽光照...,姓名: 尋路犬,評論: 清境素雅的佛寺，金世界紀錄鋼製坐姿千手觀音，路過終於好奇來參觀，真的很大。有震撼感，...,姓名: Franco Li,評論: 莊嚴神聖的千手千眼觀世音菩薩 聖像，而且還是列入金氏世界紀錄，最大鋼製雕塑。\n這裡...,...,姓名: elva sun,評論: 2023/05/01\n全台最大的千手觀音\n喜歡這個非常寧靜的地方,姓名: マサミ,評論: 因為很早到園區、所以只有在外面拍攝、六日才有開放參觀，還在建設中、期待完工之後再來，...,姓名: Chiaos Chen,評論: 園區已興建多年，千手千眼觀音的銅像已大致完成還獲得金氏世界紀錄，位於半山上環境很清幽...,姓名: Chen禎祥,評論: 只有週末跟國定假日10-16開放別跟我一樣白跑一趟,姓名: 張丰魁,評論: 藝術究竟是什麼？\n獲得金氏世界紀錄的千手千眼觀世音菩薩聖像在今年的過年期間，將室內...
1,姓名: Stella Chiu,評論: 每年櫻花盛開都會吸引大批人潮包含附近公司員工在此野餐，河景搭上櫻花彷彿日本般的景色，...,姓名: Brenda Liang,評論: 從車水馬龍的新台五路上的康誥溪一個右轉往山的方向前進就看到溪邊步道，兩旁種著滿滿的櫻...,姓名: 甜蜜蜜,評論: 康誥坑溪櫻花大道...櫻花沒有同時開花..一區一區的花開約1成至七成...二排櫻花沿...,姓名: 林建宏,評論: 1/18 大概只有開 2成 櫻花....但沒來過就隨意拍一下\n2/4 大概有5-6...,姓名: hsinjr hu,評論: 花期大多在農曆年前後，所在位置交通方便，就在省道路邊，汐科火車站走路10分鐘到。,...,姓名: Kenny,評論: 每年櫻花季的追櫻景點。汐科站新台五路方向前進，沿著康誥坑溪一路欣賞。開車的話停車可能...,姓名: KING (KING),評論: 綱路上某新聞說「盛開」！趕去一看開了1/100左右！\n等了好幾年要來賞「汐止櫻花」...,姓名: 林淑滿,評論: 河堤整理的很乾淨，是個賞櫻花的好景點，每年大約1月的月底是最佳賞櫻期,姓名: Wen-Che Eric Wu (吳文哲),評論: 2023-01-29 Sun\n離汐科站很近的賞櫻景點，約一兩百公尺，七八成開花，大...,姓名: 戲曲莉芳,評論: 位在廟後方，有洗手間，公園範圍很小，真的5分鐘走完，唯一可拍的地方（如圖），拍起來蠻...
2,姓名: Ben 乃碩,評論: 可以網路預約，也可以試試看現場排隊。\n汽車可以停在市民廣場的停車場。走路過來約十分...,姓名: 麵麵kiwi,評論: 可以免費帶寶寶來玩的好地點！有很多適合寶寶的玩具，我是帶9個多月的寶寶平日去！\n剛...,姓名: Wei Peng,評論: 4歲3個月的孩子在這裡度過快樂的早上\n如果來不及網路預約建議9:30以前來排現場號...,姓名: 黃凱楠,評論: 空間寬敞舒適，有飲水機/哺乳室與大量的玩具，現場工作人員親切和藹，很適合帶小孩來玩,姓名: Peri Tsai,評論: 玩具銀行適合學齡前小孩，充滿歡笑聲，木球池深受小朋友們喜愛，小朋友圍著木頭恐龍，搬運...,...,姓名: 李鴻偉,評論: 可免費入館，有大量玩具的空間，可玩到各式玩具，非常良善的地方，全年齡可入場，所以室內...,姓名: didi huang,評論: 20190223。\n1、非常乾淨。\n2、玩具非常多。\n3、空間寬敞。\n4、洗...,姓名: P T Y,評論: 適合和孩子一起玩許多玩具的地方，而且玩具大多數都是木質，有質感的，不是廉價劣質的塑膠...,姓名: 張明珠,評論: 多種玩具讓兒童可以動手玩樂，又是免費的，適合全家大小遊樂的地方,姓名: Anja Wu,評論: 環境很好 乾淨 遊具多元\n又是免費的\n但一般民眾自由入場只限三 六.日\n其他時...
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,姓名: PGT,評論: 聖誕節的週末跟朋友約去看山海天人展，預先買了門票，本來想說聖誕節出遊人潮到處都很多，...,姓名: elva sun,評論: 2024/02/18\n一個寓教於樂的地方\n相信我\n請一定要搭配導覽\n不然你會...,姓名: Alex Yun,評論: 全館一票通，$300，目前特展：山海天人-我在自然裡\n常設展模型跟各種展品及介紹相...,姓名: YoC Chen,評論: 常設展 全票200\n特展 全票300(常設展+特展也是這個價)\n購買實境解謎遊戲...,姓名: Mia,評論: 是想看山海天人展覽去的，結果收穫最多確實宗教展，展覽時間建議抓三小時可以慢慢看，細細...,...,姓名: Twiz,評論: （2023/11/16）\n展廳中，世界八大宗教的歷史及珍貴文物生動呈現，音樂環繞在...,姓名: Edward,評論: 太神了！裡面的氛圍、設計真的都太神了！\n每個規劃設計都很有它的意義！\n這博物館可...,姓名: chi chi,評論: 這是個很特別的博物館，除了一般常態展以外，還有額外的特展。這邊每個工作人員都很親切，...,姓名: Celine,評論: 搭配實境解謎遊戲很有趣，博物館人數不多可以細細品味，遊戲設計有巧思，環境也十分事宜，...,姓名: 潘芸柔（小小）,評論: 館藏豐富，解說用心的優質博物館\n\n博物館位在6、7樓，入場有水幕、朝聖之道\n7...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,姓名: 林亭宇,評論: 從土窯的設置、地瓜和竹筒飯的擺設開始，還會提供各式各樣的肉可以烤，在地瓜烤完之前就能...,姓名: 蕭宏德,評論: 這是個很棒的團體及家庭共同烤肉聚會的好地點，裡頭工作人員很多，都會主動幫忙，食材都是...,姓名: Cypress Lo,評論: 第一次來空間還蠻空曠的，有溜滑梯讓小孩子玩也有草皮可以踢球廁所也蠻方便的外面可以停車...,姓名: 邱近平,評論: 有得吃 有得玩 帶小孩來放電的好地方。尤其導覽的大哥講解很清楚、服務態度又很好，又很...,姓名: 張路人戊（路人戊）,評論: 說大不大說小不小的園區。服務人員很專業速度快。幫烤的地瓜也好吃。不熱的天氣來吹風呼吸...,...,姓名: 林書瑋,評論: 蠻不錯的體驗，服務人員親切，烤地瓜跟棉花糖好好玩,姓名: 沈啟鴻,評論: 這次參加的是八人份烤肉加土窯組合，再外加一隻土窯雞。烤肉食材還不錯，肉類非常多，都算...,姓名: 楊季倉,評論: 很好玩的地瓜土窯方式、還有豐富的烤肉、是很棒的親子活動．,姓名: 楊小秋,評論: 很不錯的家庭聚會的地方可以烤地瓜，放輕鬆，喝茶玩耍,姓名: 李姵蓉,評論: 非常適合親友一同出遊的地點，園區規劃良好，服務人員親切且熱心，馬上推薦給朋友～！
193,姓名: 凃昭源Norman,評論: ⚠️新北市民免費入園參觀\n.\n距離九份老街車程約15分鐘\n外圍停車場不大\n建...,姓名: golden Ya,評論: 園區雖然不大，還要爬樓梯蠻累，但能摸到四億多的金磚，覺得要發了～,姓名: 柯糾,評論: 附近停車不易，建議搭大眾運輸來，園區本身蠻多場館可以逛，不一定要付錢進來參觀，黃金博...,姓名: Feng-Yi Li,評論: 進入園會先看到遊客服務中心，前身是金瓜石車站舊址，這邊可以獲取旅遊資訊也可以讓遊客免...,姓名: Diana Lee,評論: Must-Visit Attractions in Taiwan✨\n這裡是台灣必訪...,...,姓名: Joy Chang,評論: 園區超大，只是走走不用錢！\n如果要看超大黃金80元門票，要淘金100元，礦坑體驗5...,姓名: 林建宏,評論: 要去 神社 ...路過隨拍一下📸\n下次有機會再買票參觀一下,姓名: 陳元平,評論: 週日早上前往，人潮不多\n新北市民免費入園\n展示了很多岩石相關的資訊\n國小的孩子...,姓名: F C,評論: 園區雖然不大但能摸到四億元的金磚已經很划算，建議加購50元的坑道參觀券，還有提供安全...,姓名: 佑,評論: 園區很大，適合全家來走走\n裡面有超商、還有其他販賣飲食、便當、簡餐店家\n但是周邊...
194,姓名: 立展,評論: 是個放鬆和可以讓小朋友寵物奔跑的草原，是個可以野餐和遊戲的區塊，交通而言也挺方便，公...,姓名: Benno Lee,評論: 板橋435藝文特區，是位於中華民國新北市板橋區中正路435號的藝文場所。占地約5公頃...,姓名: 彪姊,評論: 1.附近停車場多，門口的管的寬停車場，半小時25元，一小時50元。\n2.玩具博物館...,姓名: 林柔羽,評論: 這裡前身

In [25]:
df1.to_csv("newtapei_comments_1.csv", encoding = "UTF-8-sig")

In [7]:
try:
    all_comments = []
    for name in datas["shopName"]:
        if name == "TVBS大樓" :
            com_list = [name, "無評價"]
        else:        
            while True:
                try:
                    com_list = for_comments(name)
                    if com_list[2:] == []:
                        com_list = for_comments(name)
                except Exception as e:
                    if chrome != None:
                        chrome.quit()
                    com_list = for_comments(name)
                    if com_list[2:] == []:
                        com_list = for_comments(name)
                else:
                    break
        all_comments.append(com_list)
        time.sleep(2)
    print(pd.DataFrame(all_comments))          
except Exception as e:
    print(e)

千手千眼觀世音菩薩聖像SuccessFul!
name 'chrome' is not defined


In [91]:
pd.DataFrame(all_comments)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,新北投車站,姓名: 藍天,評論: 這裡蠻不錯喔，把舊的火車站保存起來，讓我們懷念一下，很棒耶！\n而且這裡也蠻美的，很...,姓名: Ruby,評論: 在新北投1號出口出站後左轉可以到復興公園泡腳，右轉則可以去旁邊的一間博物館看看裡面有...,姓名: mike hung,評論: 百年木造車站，旁邊擺放一節由新潟縣工廠打造的車廂，非常有意思，對照旁邊的新北投捷運站...,姓名: Ethan Chou,評論: 台鐵最美的車站，永遠不會誤點的車站，可以任意在鐵軌上拍照，走鐵軌，孩童嬉戲奔跑的車站...,姓名: tpklaus huang,...,"姓名: 曾國賢（Zeng, Guo-Xian）",評論: 有賣一些酷酷的東西，填問卷送一張小貼紙。,姓名: Kimi Kim,評論: 擁有豐富的溫泉資源和迷人的自然景觀，新北投車站是台北市中不可錯過的寶藏。一踏上車站，...,姓名: 石帛洋,評論: 舉辦活動有好聽得歌曲和市集可以逛\n唯一的缺點是不能進去參觀火車,姓名: 小卡的美麗日誌（Monica Lee）,評論: 整個充滿溫泉渡假氣息的車站，連新北投線的捷運也是木質色系的車廂，我覺得蠻不錯的,姓名: QUINYU,評論: 一樓有時候有市集\n這邊車廂及車站裝置藝術都是設計過\n北投熱門景點一日遊搭到這裡很...
